In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#Author: Selvaria
#keras分析期货值和平均油价联系

import pandas
import pymysql
import datetime
from dateutil.relativedelta import relativedelta

class FuturePredict(object):  # 针对时间和期货值的预测模型

    def __init__(self, code, o_type, month=2, end_time=datetime.datetime.now()):
        #self.date = date
        self.end_time = end_time
        self.code = code
        self.o_type = o_type
        self.month = month

    def get_futures_df(self):  # 获取对应编号的期货值dataframe，时间格式为字符串
        connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='cxd_data',
                                     charset='utf8')
        sql = "select * from `oil_futures_daily` where `name` = '{}' ORDER BY `time_update` ASC".format(self.code)
        df = pandas.read_sql(sql, connection)
        connection.close()
        df['time_update'] = pandas.to_datetime(df['time_update'])
        for index in df.index:
            df.loc[index, 'time_update'] = df.loc[index, 'time_update'].strftime('%Y-%m-%d')
        df_sc = df[['name', 'close_val_yesd', 'time_update']]
        return df_sc

    def get_factory_df(self, copy=3):  # 获取价格数据，复制所需的份数，时间格式为字符串
        #datetime_now = datetime.datetime.now()
        date_begin = self.end_time.strftime('%Y-%m-%d')[:-2]
        date_sql_start = date_begin + '01'
        connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='cxd_data',
                                     charset='utf8')
        sql = "SELECT `type`,`date`,avg(`price`) FROM out_price_factory_value v JOIN out_price_factory_key k ON k.`id` \
                        = v.ref_key_id WHERE `date` > '{0}' AND `source_id` = '3' AND k.`type` like '{1}%' AND \
                        GROUP BY `date`".format(date_sql_start, self.o_type)
        # print(sql)
        df_p = pandas.read_sql(sql, connection)
        connection.close()
        datetime_now = datetime.datetime.now()
        for index in df_p.index:
            # df_p.loc[index,'n_time'] = (df_p.loc[index, 'date'] - date_begin).days
            df_p.loc[index, 'date'] = df_p.loc[index, 'date'].strftime('%Y-%m-%d')
            df_p.loc[index, 'n_time'] = (
                    datetime.datetime.strptime(df_p.loc[index, 'date'], '%Y-%m-%d') - date_begin).days
        return df_p

    def get_former_f(self, date):  # 获取缺失的期货值
        df_result = self.get_futures_df()
        df_result.rename(columns={'time_update': 'date'}, inplace=True)
        datetime_date = datetime.datetime.strptime(str(date), '%Y%m%d')
        last_date = datetime_date - datetime.timedelta(days=1)  # 前1天的日期，datetime格式
        last_date1 = datetime_date + datetime.timedelta(days=1)  # 后1天的日期，datetime格式
        last_date2 = datetime_date - datetime.timedelta(days=2)  # 前2天的日期，datetime格式
        last_date3 = datetime_date - datetime.timedelta(days=3)  # 前3天的日期，datetime格式
        last_date4 = datetime_date - datetime.timedelta(days=4)  # 前4天的日期，datetime格式
        last_date9 = datetime_date - datetime.timedelta(days=9)  # 前9天的日期，datetime格式
        for index in df_result.index:
            df_result.loc[index, 'date'] = int(
                datetime.datetime.strptime(df_result.loc[index, 'date'], '%Y-%m-%d').strftime('%Y%m%d'))
        try:
            futures = df_result['close_val_yesd'][df_result['date'] == date].values[0]
            return futures
        except:
            try:
                futures = df_result['close_val_yesd'][df_result['date'] == int(last_date.strftime('%Y%m%d'))].values[0]
                return futures
            except:
                try:
                    futures = df_result['close_val_yesd'][df_result['date'] == int(last_date1.strftime('%Y%m%d'))].values[0]
                    return futures
                except:
                    try:
                        futures = \
                        df_result['close_val_yesd'][df_result['date'] == int(last_date2.strftime('%Y%m%d'))].values[0]
                        return futures
                    except:
                        try:
                            futures = \
                            df_result['close_val_yesd'][df_result['date'] == int(last_date3.strftime('%Y%m%d'))].values[0]
                            return futures
                        except:
                            try:
                                futures = \
                                df_result['close_val_yesd'][df_result['date'] == int(last_date4.strftime('%Y%m%d'))].values[
                                    0]
                                return futures
                            except:
                                try:
                                    futures = df_result['close_val_yesd'][
                                        df_result['date'] == int(last_date9.strftime('%Y%m%d'))].values[0]
                                    return futures
                                except:
                                    print('%s日的期货值为最近20天的平均值' %date)
                                    futures = self.get_avg_futures(date)
                                    return futures

    def df_handle(self):  # dataframe结构处理，合并有用的值，用于模型训练
        df_f = self.get_futures_df()
        df_f.rename(columns={'time_update': 'date'}, inplace=True)
        df_p = self.get_factory_df()
        df_result = pandas.merge(df_f, df_p, on='date', how='right')
        df_nan = df_result[df_result.isnull().close_val_yesd == True]
        for index in df_nan.index:
            date_int = int(datetime.datetime.strptime(df_nan.loc[index, 'date'], '%Y-%m-%d').strftime('%Y%m%d'))
            val_x = self.get_former_f(date_int)
            # print(val_x)
            df_nan.loc[index, 'close_val_yesd'] = val_x
        df_merge = df_result[df_result['close_val_yesd'] > 10]
        df_h = df_merge.append(df_nan)
        df_result = df_h[df_h['close_val_yesd'].notnull()]
        return df_result
    
fp = FuturePredict('广悦化工','SC1901','0#国五')
df = fp.df_handle()
df

In [5]:
import pandas as pd

df = pd.DataFrame({'A':['A0', 'A1', 'A2', 'A3'],
                   'B':['B0', 'B1', 'B2', 'B3'],
                   'C':['C0', 'C1', 'C2', 'C3'],
                   'D':['D0', 'D1', 'D2', 'D3']})

df2 = df.append(df, ignore_index=True)
df3 = df2.append(df, ignore_index=True)
df3

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A0,B0,C0,D0
5,A1,B1,C1,D1
6,A2,B2,C2,D2
7,A3,B3,C3,D3
8,A0,B0,C0,D0
9,A1,B1,C1,D1
